# Club elo scraper
Store club elo per every 6 months as list of pandas dataframes
and store it as a csv file

In [23]:
from datetime import datetime
import io
import requests
import pandas as pd
from pathlib import Path

In [28]:
# Generate dates with 6-month intervals starting from 2010-01-01 to today
date_range = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq='6M')

# Convert the dates to the desired string format 'YYYY-MM-DD'
date_list = date_range.strftime('%Y-%m-%d').tolist()

/var/folders/nh/kbd4vyw93l9_cfg59h74rpwh0000gn/T/ipykernel_7772/3712119244.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq='6M')


['2020-01-31',
 '2020-07-31',
 '2021-01-31',
 '2021-07-31',
 '2022-01-31',
 '2022-07-31',
 '2023-01-31',
 '2023-07-31',
 '2024-01-31',
 '2024-07-31']

In [29]:

dfs = []
merged_df = pd.DataFrame()

for date in date_list:
    response = requests.get(f"http://api.clubelo.com/{date}")

    club_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    club_elo_df['Rank'] = club_elo_df.index + 1
    club_elo_df['Updated'] = date
    
    dfs.append(club_elo_df)
    # merged_df.append(club_elo_df)
    
res = pd.concat(dfs)



In [40]:

res = pd.concat(dfs)
res.head()

,Rank,Club,Country,Level,Elo,From,To,Updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


In [41]:
# Now we want to change club name according to clubs.csv and also change the index column name
# columns = [x.lower() for x in res.columns]
new_rows = ['rank', 'name', 'country', 'level', 'elo', 'from', 'to', 'updated']

res.columns = new_rows
# res = res[0:].reset_index(drop=True)  # Remove the first row and reset index
# 
res.head()


,rank,name,country,level,elo,from,to,updated
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31


In [42]:
res['club_id'] = -1

In [43]:
#

,rank,name,country,level,elo,from,to,updated,club_id
0,1,Liverpool,ENG,1,2088.261963,2020-01-30,2020-02-01,2020-01-31,-1
1,2,Man City,ENG,1,2002.820190,2020-01-22,2020-02-02,2020-01-31,-1
2,3,Barcelona,ESP,1,1964.985352,2020-01-26,2020-02-02,2020-01-31,-1
3,4,Bayern,GER,1,1951.214600,2020-01-26,2020-02-01,2020-01-31,-1
4,5,Juventus,ITA,1,1908.358643,2020-01-27,2020-02-02,2020-01-31,-1


['rank', 'club', 'country', 'level', 'elo', 'from', 'to', 'updated']

In [ ]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

res.to_csv(DATA_DIR / 'club_elos.csv')